In [3]:
# importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import tensorflow.keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from sklearn.model_selection import train_test_split

In [2]:
pip install keras-tuner

     |████████████████████████████████| 133 kB 24.6 MB/s 


In [3]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [4]:
import tensorflow as tf
from tensorflow import keras

(train_X,train_Y), (test_X,test_Y) = keras.datasets.fashion_mnist.load_data()
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

4431872/4422102 [==============================] - 0s 0us/step
Training data shape :  (60000, 28, 28) (60000,)
Testing data shape :  (10000, 28, 28) (10000,)


In [5]:
def build_model(hp):  
    model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # Sets minimum and max values of filters from 32 to 128
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    tensorflow.keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    tensorflow.keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        activation='relu'
    ),
    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    tensorflow.keras.layers.Dense(10, activation='softmax')
  ])
    
    model.compile(optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [6]:
tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=10,directory='output',project_name="Mnist Fashion")

# Random Search will tell which parameter is best for Keras tuning

In [10]:
train_X_new=train_X/255.0
test_X_new=test_X/255.0

In [8]:
train_X_new[0].shape

(28, 28)

In [11]:
train_X_new=train_X_new.reshape(len(train_X_new),28,28,1)
test_X_new=test_X_new.reshape(len(test_X_new),28,28,1)

In [10]:
tuner_search.search(train_X_new, train_Y,epochs=15,validation_split=0.1)

Trial 10 Complete [00h 01m 57s]
accuracy: 0.8926296234130859

Best accuracy So Far: 0.9950740933418274
Total elapsed time: 00h 23m 38s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        41520     
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 64)        27712     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 64)                1982528   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,053,370
Trainable params: 2,053,370
Non-

In [15]:
model.fit(train_X_new, train_Y, epochs=50, validation_split=0.1, initial_epoch=15)

Epoch 16/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0099 - accuracy: 0.9973 - val_loss: 0.8780 - val_accuracy: 0.9042
Epoch 17/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0088 - accuracy: 0.9977 - val_loss: 1.0420 - val_accuracy: 0.9118
Epoch 18/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0143 - accuracy: 0.9962 - val_loss: 0.9435 - val_accuracy: 0.9045
Epoch 19/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0095 - accuracy: 0.9973 - val_loss: 1.1955 - val_accuracy: 0.8973
Epoch 20/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0108 - accuracy: 0.9968 - val_loss: 1.0867 - val_accuracy: 0.9068
Epoch 21/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0106 - accuracy: 0.9970 - val_loss: 1.0778 - val_accuracy: 0.9120
Epoch 22/50
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0114 - accuracy: 0.9969 - val_loss: 0.9899 -

In [16]:
test_eval = model.evaluate(test_X_new, test_Y, verbose = False)
print('Test accuracy:', test_eval[1])

Test accuracy: 0.906499981880188


In [6]:
    model2 = tensorflow.keras.Sequential([
        tensorflow.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
               input_shape=(28, 28, 1)),
        tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tensorflow.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        tensorflow.keras.layers.BatchNormalization(),        
        tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        tensorflow.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        tensorflow.keras.layers.BatchNormalization(),
        tensorflow.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
        tensorflow.keras.layers.BatchNormalization(),    
        tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2)),   
        
        tensorflow.keras.layers.Flatten(),
        
        tensorflow.keras.layers.Dense(1024, activation='relu'),
        
        tensorflow.keras.layers.Dense(512, activation='relu'),
        
        tensorflow.keras.layers.Dense(10, activation='softmax')
    ])

In [8]:
from tensorflow.keras.optimizers import Adam
adam = Adam(lr=0.0001, decay=1e-6)
model2.compile(optimizer=adam,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
model2.fit(train_X_new, train_Y, epochs=20, validation_split=0.1)

Epoch 1/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0397 - accuracy: 0.9857 - val_loss: 0.3146 - val_accuracy: 0.9220
Epoch 2/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0358 - accuracy: 0.9866 - val_loss: 0.4079 - val_accuracy: 0.9127
Epoch 3/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0323 - accuracy: 0.9885 - val_loss: 0.3396 - val_accuracy: 0.9218
Epoch 4/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0282 - accuracy: 0.9901 - val_loss: 0.3980 - val_accuracy: 0.9210
Epoch 5/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0284 - accuracy: 0.9899 - val_loss: 0.3640 - val_accuracy: 0.9245
Epoch 6/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0278 - accuracy: 0.9902 - val_loss: 0.4058 - val_accuracy: 0.9160
Epoch 7/20
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0227 - accuracy: 0.9919 - val_loss: 0.3915 - val_ac

In [15]:
test_eval = model2.evaluate(test_X_new, test_Y, verbose = False)
print('Test accuracy:', test_eval[1])

Test accuracy: 0.9225000143051147
